# GPT-3 Blog Title Optimizer Demo

Here's a demo of how to leverage GPT-3 to build a title optimizer, complete with many examples!

For a walkthrough into what the code is doing, see [the walkthrough Notebook](gpt3_title_optimizer_walkthrough.ipynb).

This notebook assumes you have a `OPENAI_API_KEY` specified in a `.env` file, e.g.

```properties
OPENAI_API_KEY="<FILL IN>"
FINETUNED_MODEL="<FILL IN>"
```

In [1]:
import os
import re
from math import exp

import openai
import pandas as pd
from IPython.display import display, HTML
from dotenv import load_dotenv

load_dotenv()

assert os.getenv("OPENAI_API_KEY"), "No OPENAI_API_KEY defined in .env."
assert os.getenv("FINETUNED_MODEL"), "No FINETUNED_MODEL defined in .env."

openai.api_key = os.getenv("OPENAI_API_KEY")


In [6]:
def optimize_title(title_input, temperature=1.0):
    base_prompt = "Rewrite the following blog post title into six different titles but optimized for social media virality: {0}\n\n-"
    finetune_prompt = "Title: {0} ->"

    r = openai.Completion.create(
    model="text-davinci-002",
    prompt=base_prompt.format(title_input),
    temperature=temperature,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

    gen_titles = re.split(r" ?\n-", r["choices"][0]["text"])

    gen_titles = list(set(gen_titles + [title_input]))

    ranked_titles = []

    for gen_title in gen_titles:
        r = openai.Completion.create(
            model=os.getenv("FINETUNED_MODEL"),
            prompt=finetune_prompt.format(gen_title),
            temperature=0,
            max_tokens=1,
            logprobs=1,
        )

        title_class = r["choices"][0]["text"]
        class_prob = exp(r["choices"][0]["logprobs"]["token_logprobs"][0])
        if title_class == " bad":
            class_prob = 1.0 - class_prob

        ranked_titles.append(
            (
                f"<strong>{gen_title}</strong>" if gen_title == title_input else gen_title,
                class_prob,
            )
        )

    df = pd.DataFrame(ranked_titles, columns=["Title", "Good Prob"])
    df = df.sort_values(by="Good Prob", ascending=False)


    display(
        HTML(
            df.to_html(
                formatters={"Good Prob": lambda x: f"{x:.1%}"}, escape=False, index=False
            )
        )
    )


## Running My Blog Posts Through The Optimizer

First up, the original name of the blog post about this project, **Creating a Blog Post Title Optimizer by Finetuning GPT-3 on Hacker News**:

In [10]:
optimize_title("Creating a Blog Post Title Optimizer by Finetuning GPT-3 on Hacker News", temperature=0)

Title,Good Prob
Optimize your Blog Post Titles for More Clicks with GPT-3,51.1%
GPT-3: The Key to Optimizing your Blog Post Titles,41.9%
How to Optimize your Blog Post Titles using GPT-3,29.9%
Get More Clicks: Use GPT-3 to Optimize your Blog Post Titles,23.3%
The Secret to Optimizing your Blog Post Titles? GPT-3,16.9%
Creating a Blog Post Title Optimizer by Finetuning GPT-3 on Hacker News,14.8%


If `temperature` is nonzero, then the generated titles will be unique each time you run it.

In [11]:
optimize_title("Creating a Blog Post Title Optimizer by Finetuning GPT-3 on Hacker News", 0.7)

Title,Good Prob
How to Create a Blog Post Title Optimizer,40.6%
Creating a Blog Post Title Optimizer with GPT-3,22.8%
Creating a Blog Post Title Optimizer by Finetuning GPT-3 on Hacker News,14.8%
Creating a Blog Post Title Optimizer for Free,14.3%
Creating a Blog Post Title Optimizer in Minutes,13.2%
Creating a Blog Post Title Optimizer without any Code,12.8%
Creating a Blog Post Title Optimizer with only 3 Lines of Code,9.5%


In [13]:
optimize_title("Creating a Blog Post Title Optimizer by Finetuning GPT-3 on Hacker News", 1.0)

Title,Good Prob
The Best Way to Optimize Your Blog Post Titles,40.3%
How to Create a Blog Post Title Optimizer with GPT-3,32.0%
How to Optimize Your Blog Post Titles with GPT-3,25.5%
Get More Clicks: How to Optimize Your Blog Post Titles with GPT-3,23.8%
Creating a Blog Post Title Optimizer by Finetuning GPT-3 on Hacker News,14.8%
How to Optimize Your Blog Post Titles for Virality,13.9%


Most of them have a low probability of being good, which in retrospect I probably should have checked before I wrote this post.

From the third one I like `How to Create a Blog Post Title Optimizer with GPT-3` but it's important to incorporate Hacker News, so I'll test a variant of that.

In [19]:
optimize_title("How to Create a Blog Post Title Optimizer with GPT-3 and Hacker News Data")

Title,Good Prob
GPT-3 and Hacker News Data: How to Optimize your Blog Post Title,41.8%
Blog Post Title Optimizer: GPT-3 and Hacker News Data,29.2%
How to Create a Blog Post Title Optimizer with GPT-3 and Hacker News Data,20.8%
The Best way to Optimize your Blog Post Title: GPT-3 and Hacker News,19.7%
Optimizing your Blog Post Title with GPT-3 and Hacker News Data,16.5%
How to Optimize your Blog Post Title for GPT-3 and Hacker News,16.0%
How to Use GPT-3 and Hacker News Data to Optimize your Blog Post Title,14.6%


There is a performance hit (32.0% to 20.8%) but it's important so I'll leave it, and the variants to that are not better.

A few more demos:

In [20]:
optimize_title("Absurd AI-Generated Professional Food Photography with DALL-E 2")

Title,Good Prob
DALL-E 2's absurd AI-generated professional food photography,69.6%
See the professional food photography DALL-E 2's AI created,52.9%
DALL-E 2: Now with AI-generated professional food photography!,46.6%
Professional food photography gets even more absurd with DALL-E 2,41.3%
"DALL-E 2's professional food photography, now with AI!",39.8%
Absurd AI-Generated Professional Food Photography with DALL-E 2,29.3%
The AI-generated professional food photography of DALL-E 2,27.0%


In [21]:
optimize_title("How to Generate Customized AI Art Using VQGAN and CLIP")

Title,Good Prob
VQGAN and CLIP: How to Generate AI Art,58.9%
Design Your Own AI Art With VQGAN and CLIP,43.4%
How to Create AI Art Using VQGAN and CLIP,42.1%
Make Custom AI Art With VQGAN and CLIP,41.8%
How to Generate Customized AI Art Using VQGAN and CLIP,34.0%
Creating AI Art With VQGAN and CLIP,32.3%
How to Use VQGAN and CLIP for Custom AI Art,11.5%


In [22]:
optimize_title("Things About Real-World Data Science Not Discussed In MOOCs and Thought Pieces")

Title,Good Prob
The dark side of data science MOOCs,67.4%
What data science MOOCs don't tell you,33.6%
The 5 things data science MOOCs don't prepare you for,31.3%
5 Important things data science MOOCs don't teach you,17.7%
Things About Real-World Data Science Not Discussed In MOOCs and Thought Pieces,16.9%
5 things you won't learn about data science from online courses,13.7%
5 things data science thought pieces won't tell you,9.4%


In [23]:
optimize_title("Visualizing One Million NCAA Basketball Shots")

Title,Good Prob
What one million NCAA basketball shots reveals about the game,82.2%
What patterns can be found in one million NCAA basketball shots?,79.5%
One million NCAA basketball shots analyzed: what we learned,76.7%
How studying one million NCAA basketball shots can change the way we see the game,75.1%
One million NCAA basketball shots: what do they tell us?,73.1%
One million NCAA basketball shots: the patterns and what they mean,68.1%
Visualizing One Million NCAA Basketball Shots,67.7%


In [24]:
optimize_title("Leaving Apple Inc.")

Title,Good Prob
Leaving Apple Inc.,91.4%
A key player is leaving Apple Inc.,87.0%
Apple Inc. is losing a valuable employee,86.6%
Employee leaving Apple Inc. could mean big changes,73.5%
Apple Inc. in trouble as employee leaves,71.4%
Could this be the end of Apple Inc.? Employee leaves,68.1%
Apple Inc. suffers blow as employee leaves,55.7%


In [25]:
optimize_title("What Percent of the Top-Voted Comments in Reddit Threads Were Also 1st Comment?")

Title,Good Prob
Surprising Stats on Top Comments in Reddit Threads,44.9%
Incredible Stats on Top Comments in Reddit Threads,42.9%
Astonishing Facts About Top Comments in Reddit Threads,34.5%
What Percent of Top-Voted Comments in Reddit Threads Were 1st Comment?,33.4%
What Percent of the Top-Voted Comments in Reddit Threads Were Also 1st Comment?,29.8%
You Won't Believe How Many Top-Voted Comments in Reddit Threads Were 1st Comment,11.8%


In [26]:
optimize_title("Movie Review Aggregator Ratings Have No Relationship with Box Office Success")

Title,Good Prob
How Movie Review Aggregator Ratings Impact Box Office Success,21.6%
Movie Review Aggregator Ratings Have No Relationship with Box Office Success,15.3%
Why Movie Review Aggregator Ratings Have No Relationship with Box Office Success,13.5%
The Real Truth About Movie Review Aggregator Ratings and Box Office Success,10.3%
Do Movie Review Aggregator Ratings Really Matter for Box Office Success?,7.3%
What Movie Review Aggregator Ratings Can Tell Us About Box Office Success,6.8%
Do Better Movie Review Aggregator Ratings Mean More Box Office Success?,3.6%


In [27]:
optimize_title("How to Visualize New York City Using Taxi Location Data and ggplot2")

Title,Good Prob
How to Visualize New York City Using Taxi Location Data and ggplot2,51.1%
See New York City Like Never Before With This Taxi Location Data Visualization,29.8%
How to Use Taxi Location Data and ggplot2 to Visualize New York City,23.4%
This ggplot2 Visualization of New York City Will Blow Your Mind,22.1%
A New Way to Look at New York City: Visualizing Taxi Location Data,17.1%
Here's How to Visualize New York City Using Taxi Location Data,17.0%
You Won't Believe What Taxi Location Data Reveals About New York City,12.6%


In [28]:
optimize_title("Why is the Most-Viewed Gaming Video on YouTube About Cars 2?")

Title,Good Prob
Why is the Most-Viewed Gaming Video on YouTube About Cars 2?,34.7%
The Strange Case of the Most-Viewed Gaming Video on YouTube,32.7%
What do Cars 2 and Gamers Have in Common?,30.4%
The Most-Viewed Gaming Video on YouTube will Surprise You!,18.9%
Why the Gaming World is Obsessed with Cars 2,13.3%
Cars 2 is the Most-Viewed Gaming Video on YouTube- But Why?,12.4%


In [29]:
optimize_title("When Uninstalling A PC Game Erases the Entire Hard Drive")

Title,Good Prob
How uninstalling a PC game can erase your hard drive,66.8%
Beware! Uninstalling this PC game deletes your hard drive,62.0%
Uninstalling a PC game and losing your entire hard drive,60.1%
When uninstalling a PC game erases your entire hard drive,59.1%
What happens when you uninstall a PC game? This.,47.0%
When Uninstalling A PC Game Erases the Entire Hard Drive,30.5%
Uninstalling PC game? Here's what you need to know about losing your data,10.4%
